# Timeseries: CMEMS OSTIA Daily SST Data in CSV

<div class="alert alert-info" role="alert">

We will be accessing 18 years of daily sea surface temperature measurements from the Copernicus Marine Service (CMEMS):
https://marine.copernicus.eu

</div>

#### Importing the python libraries you'll need: basic setup

In [ ]:
# Data Handling and Manipulation
import os
import numpy as np
import pandas as pd

# Data Visualisation and Plotting
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.dates import DateFormatter, YearLocator
import seaborn as sns

# Apply Statistical Testing
from scipy.stats import linregress
import pymannkendall as mk

# Supress warnings (not errors)
import warnings
warnings.filterwarnings('ignore')

#### It looks like we don't have all the libraries we want by default, so let's install what we need:
`pip install pymannkendall`

<div class="alert alert-info" role="alert">

You just used coordinated execution of a task between the Windows OS and your python environment using BASH code! Nice work!

</div>


### 
### Section1: Reading in the Sea Surface Temperature data

In [ ]:
pwd # This will output your current path, which you will need to copy and paste into the next cell

#### **NB** To the `cell below`, you need to add `your path` from the `cell above` (pwd) and `your .nc filename`

In [ ]:
PATHWAY = os.path.join('YOUR PATH\\YOUR PATH') #Replace with your pathway from pwd
IN_FILE = "YOUR DATA FILE.txt"                 #Replace with your filename from ls

In [ ]:
dataframe = pd.read_csv(os.path.join(PATHWAY, IN_FILE), comment='#')    #The hashtag prevents commented lines from being read as data
# Display using print
print(dataframe.head(5))

### 
### Section2: Key Steps for Data 'Readiness' (i.e.: before analysis)

##### The dates are in a common notation for `UTC` (Coordinated Universal Time) but this is a pain to work with. Let's fix things using the `pd.to_datetime` function in the `pandas` library. 
##### This function converts a column of date-like objects into `datetime64 format`, which is a datetime type that `pandas` uses for date and time manipulation. Normally, it would be worth keeping both date and time in a column called 'Date_Time', but with daily data there's no need.

In [ ]:
# The UTC format that our dates (times) are currently in:
dataframe['dates'] = pd.to_datetime(dataframe['time'], format='%Y-%m-%dT%H:%M:%S.%fZ', utc=True)

# Step1: convert to a more friendly format, add 'dates' as new column 
dataframe['dates'] = dataframe['dates'].dt.strftime('%Y-%m-%d %H:%M')

# Step2: Remove the times in this case as they contain no real information
dataframe['dates'] = pd.to_datetime(dataframe['dates'])

# Step3: Drop the now unnecessary 'time' column
dataframe = dataframe.drop(columns=['time'])

# Display
print(dataframe.head(2))

In [ ]:
# Re-order your columns so date is still first
dataframe = dataframe[['dates','analysed_sst']]
# Display
print(dataframe.head(2))

#### NEARLY perfect, but temperatures are in `Kelvin`

In [ ]:
# Make a new column in your dataframe with SST in Celsius(°C)
dataframe['SST_Celsius'] = dataframe['analysed_sst'] - 273.15

# Assign more helpful names to the columns:
dataframe.columns = ['Dates', 'SST_Kelvin', 'SST_Celsius']

# Display
print(dataframe.head(5))

#### Btw, it's now really easy to output this lovely new perfect data in a saved new CSV file of your own:

In [ ]:
dataframe.to_csv('My_New_SST_Data.csv', index=False)

### 
### Section3: Plot Timeseries of SST, including with Mean SST

#### Let's make a timeseries of SST over time from the single pixel we selected on CMEMS

In [ ]:
# How many unique years are in our time series
num_years = dataframe['Dates'].dt.year.nunique()
print(f"There are {num_years} unique years in the time series.")

In [ ]:
# Figure 1: Plot your first timeseries!
fig1, ax = plt.subplots(figsize = (16, 5), dpi = 250)

# Set major locator to show every year and format the x-axis to only show years
# When you have so many data points, this helps to ensure you can see the dates
ax.xaxis.set_major_locator(YearLocator(1))
date_form = DateFormatter('%Y')
ax.xaxis.set_major_formatter(date_form)

# Rotate the x-axis labels for even better readability:
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

# Plot the data using 's' square markers in blue
plt.plot(dataframe['Dates'], dataframe['SST_Celsius'], linestyle = '', marker='s', markersize=1, c='blue')

# Set your x and y axis limits (personal preference)
# ax.set_ylim([16, 25])
ax.set_xlim([dataframe['Dates'].min(), dataframe['Dates'].max()])

# Gridlines included
ax.grid(True)

# Add labels
plt.xlabel('Time')
plt.ylabel('Temp (°C)')

# Show the plot
plt.show()

In [ ]:
# To save any of the Figures (change XX to fig1/ fig2/ fig3/ fig4 etc)
# fig1.savefig('SST_Timeseries.png', bbox_inches='tight')

In [ ]:
# We can calculate the mean (and other stats) for plotting with numpy (np) or pandas (pd)
# Using Pandas:
Mn = dataframe['SST_Celsius'].mean()

# Print Mean SST:
print ('Mean SST off South Fuerteventura over the', num_years, 
       'timeseries is ~', "%.2f" % Mn,'°C')                   #Report to 2 decimal places using ["%.2f" %]

In [ ]:
# Figure 2: Plotting as above, but let's add a line, the mean, and some labels
fig2, ax = plt.subplots(figsize = (16, 5), dpi = 250)

# Set major locator to show every year and format the x-axis to only show years
# When you have so many data points, this helps to ensure you can see the dates
ax.xaxis.set_major_locator(YearLocator(1))
date_form = DateFormatter('%Y')
ax.xaxis.set_major_formatter(date_form)

# Rotate the x-axis labels for even better readability:
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

# Plot the data using a line in a darker blue, and add label
plt.plot(dataframe['Dates'], dataframe['SST_Celsius'], linestyle='-',linewidth=1, c='mediumblue', label='Sea Surface Temperature')

# Set your x and y axis limits to remove spaces (your preference)
ax.set_ylim([17, 25])
ax.set_xlim([dataframe['Dates'].min(), dataframe['Dates'].max()])

# Plot the mean line
plt.plot([dataframe['Dates'].min(), dataframe['Dates'].max()], 
         [Mn, Mn], 
         color = 'r', 
         linestyle = '--', 
         linewidth = 2, 
         label = f'Mean SST over ~{num_years} Years')

# In this plot, we will choose to NOT plot gridlines:
ax.grid(False)

# Add labels
plt.xlabel('Time')
plt.ylabel('Temp (°C)')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# To save any of the Figures (change XX to fig1/ fig2/ fig3/ fig4 etc)
# fig2.savefig('SST_Timeseries_plusMean.png', bbox_inches='tight')

###
### Section4: Timeseries of SST with the Addition of a Trendline
#### `Linear Regression`:

In [ ]:
# Data Prep for Linear regression:
x_date = dataframe['Dates']           # Your actual dates
x_num1 = dates.date2num(x_date)   # Convert dates to numerical format
SST = dataframe['SST_Celsius']     # Your actual SST data

# Perform linear regression
slope, intercept, r_value, p_value, std_err = linregress(x_num1, SST)

# Calculate the trendline
trendline = slope * x_num1 + intercept

In [ ]:
print('Slope: change in SST =', "%.10f" % slope,'°C per day')

In [ ]:
print('Number of unique days in our timeseries =', len(dataframe))

In [ ]:
# Plotting
fig3, ax = plt.subplots(figsize=(16, 5), dpi=250)

# Set x-axis to show years only
ax.xaxis.set_major_locator(YearLocator(1))
date_form = DateFormatter('%Y')
ax.xaxis.set_major_formatter(date_form)

# Rotate the x-axis labels for even better readability:
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

# Plot the original SST data
ax.plot(x_date, SST, 's', markersize=1, color='navy')
# Plot the trendline
ax.plot(x_date, trendline, color='r', label=f'y = {slope:.6f}x + {intercept:.2f}')

# Set your x and y axis limit (preference)
ax.set_ylim([SST.min(), SST.max()])
ax.set_xlim([x_date.min(), x_date.max()])

# Here, we will plot gridlines using a light grey (silver) dotted line (:)
ax.grid(True, color='silver', linestyle=':', linewidth=1)

# Add title, labels, and legend
plt.title(f'Trend in Fuerteventura SST Over ~{num_years} Years', fontweight='bold', color='dimgrey')
plt.xlabel('Time')
plt.ylabel('Temp (°C)')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# To save any of the Figures (change XX to fig1/ fig2/ fig3/ fig4 etc)
# fig3.savefig('SST_Timeseries_plusRegression.png', bbox_inches='tight')

----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------


### Section5: Statistical Testing for Significance
#### Is the change in SST over time significant? We can assess this this using the `Mann-Kendall` test:

#### The `Mann-Kendall` Trend Test is used to determine whether or not a trend exists in timeseries data. It is a `non-parametric` test, meaning there is no underlying assumption made about `normality`.

mk.original_test(yourdataframe)
OUTPUT-
 trend: This tells the trend-increasing, decreasing, or no trend.
 h: True if the trend is present. False if no trend is present.
 p: The p-value of the test.
 z: The normalized test statistic.
 Tau: Kendall Tau.
 s: Mann-Kendal’s score
 var_s: Variance S
 slope: Theil-Sen estimator/slope
 intercept: Intercept of Kendall-Theil Robust Line

If you are worried about autocorrelation in your data, use a modified Mann Kendall test instead, e.g.:
mk.hamed_rao_modification_test(yourdataframe)

In [ ]:
# Is this trend in SST significant?
trend1 = mk.original_test(dataframe['SST_Celsius'])
print("SST Trend is",trend1[0], ", P-value =", ("%.3f" % trend1[2]))

----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
### Extra steps for data analysis

#### Which was the hottest day and which was the coldest day in the timeseries?

In [ ]:
# Calculating max and min SST values, and determining where they sit in the timeseries using pandas:
max_sst_value = dataframe['SST_Celsius'].max()
min_sst_value = dataframe['SST_Celsius'].min()

# Finding the index of the max and min SST values
max_sst_index = dataframe['SST_Celsius'].idxmax()
min_sst_index = dataframe['SST_Celsius'].idxmin()

# Getting the dates corresponding to the max and min SST values and formatting them without time
day_max = dataframe['Dates'][max_sst_index].strftime('%Y-%m-%d')
day_min = dataframe['Dates'][min_sst_index].strftime('%Y-%m-%d')

# Printing the results
print(f'The mean SST in my timeseries is {Mn:.2f}°C, with max temp [{max_sst_value:.2f}°C at index {max_sst_index}, {day_max}] '
      f'and min temp [{min_sst_value:.2f}°C at index {min_sst_index}, {day_min}].')

#### Everyone loves a good boxplot, right?

In [ ]:
fig4, ax = plt.subplots(figsize=(4, 5))

# Set the overall style
sns.set_style('whitegrid')

# Boxplot with customized palette and width
ax = sns.boxplot(y=dataframe['SST_Celsius'], width=0.25, color='lightseagreen', flierprops=dict(marker='o', markersize=5))

# Set y-limits
ax.set_ylim([0, 30])

# Add labels for x and y axes, suppressing output with semicolons
ax.set_xlabel(f'Fuerteventura SST distribution over ~{num_years} years', fontsize=10);
ax.set_ylabel('Sea Surface Temperature (°C)', fontsize=10);

# Customize the gridlines to be dotted on the y-axis only
ax.grid(True, linestyle=':', linewidth=1, color='gray', axis='y')

# Show the plot
plt.show()

# Save the figure using the figure handle (fig1)
# fig4.savefig('SST_dist_boxplot.png', dpi=300, bbox_inches='tight')

<div class="alert alert-info" role="alert">

Your final task for this practical session is to change the `y-axis` limits to a minimum of 15°C and a maximum of 26°C, pick a new colour to replace `lightseagreen`, save the boxplot figure as a `.png`, and upload to the DLE for 5 marks.

</div>